In [1]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset
from azureml.train.hyperdrive import RandomParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice

ws = Workspace.from_config()
datastore = ws.get_default_datastore()
experiment = Experiment(workspace=ws, name='BasicModel')

workspace = Workspace.from_config()

dataset = Dataset.get_by_name(workspace, name='rx20_all_models',version="latest")

config = ScriptRunConfig(
        source_directory='./src',
        script='BasicModel.py',
        compute_target='A00238622',
        arguments=[
            '--input-data', dataset.as_named_input('rx20_all_models')],
    )

params = RandomParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        #'--n_estimators': choice(range(10, 200)),
        #'--max_depth' : choice(range(5, 32)),
        #'--min_samples_split': choice(range(2, 10,2)),
        #'--min_samples_leaf':  choice(range(1, 5,1)),
        #'--max_depth' : choice(range(5, 32)),
        '--number_of_feature_values' : choice(range(9,11))

    }
)

env = Environment.from_conda_specification(
        name='sklearn-env',
        file_path='sklearn-env.yml'
    )

config.run_config.environment = env

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=config, 
                              hyperparameter_sampling=params, 
                              policy=None, 
                              primary_metric_name='median_relative_error_test', 
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                              max_total_runs=3, 
                              max_concurrent_runs=3) 

run = experiment.submit(hyperdrive)
aml_url = run.get_portal_url()
print("Submitted to compute cluster. Click link below")
print("")
print(aml_url)

Submitted to compute cluster. Click link below

https://ml.azure.com/runs/HD_0f2086e8-65da-4231-b521-5fa088115d98?wsid=/subscriptions/ac5709b8-dd2b-46e3-98ae-bf53155e0e80/resourcegroups/nereva_rg/workspaces/e2r2s14mls0004&tid=13c728e0-bb0c-4cf7-8e10-5b327279d6d9
